In [27]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.metrics import roc_auc_score, accuracy_score
from torch.utils.data import TensorDataset, DataLoader, random_split
from torchvision import models, transforms


In [29]:
# prompt: get current working dir and I will need to download these file to rico dir at my google drive RICO folder
# curl -O https://storage.googleapis.com/crowdstf-rico-uiuc-4540/rico_dataset_v0.1/ui_details.csv
# curl -O https://storage.googleapis.com/crowdstf-rico-uiuc-4540/rico_dataset_v0.1/ui_layout_vectors.zip
# curl -O https://storage.googleapis.com/crowdstf-rico-uiuc-4540/rico_dataset_v0.1/traces.tar.gz

import os

rico_dir = '/Users/wenpeishao/Downloads/CHTC/rico/data'  # Update this path if needed

if not os.path.exists(rico_dir):
  os.makedirs(rico_dir)

os.chdir(rico_dir)


In [ ]:
# Unzip and untar the dataset files into the local working directory
!echo "Extracting dataset files..."
os.chdir('/Users/wenpeishao/Downloads/CHTC/rico/data')
!mkdir -p combined/
!tar -xzf unique_uis.tar.gz -C combined/
!tar -xzf traces.tar.gz -C combined/
!unzip -o ui_layout_vectors.zip -d combined/

# Remove zip files to free up space
!echo "Removing compressed files to free up space..."
!rm -f unique_uis.tar.gz traces.tar.gz ui_layout_vectors.zip

# Verify the extracted contents
!echo "Listing contents of the combined directory..."
!ls -la combined/

In [35]:
# Define paths based on the bash script setup
ui_details_path = './ui_details_updated.csv'  # CSV file containing UI details and labels
images_dir = './combined/combined/'   # Directory containing the UI images



/Users/wenpeishao/Downloads/CHTC/rico/data


In [32]:
# Load the UI details
ui_details_df = pd.read_csv(ui_details_path)
print(f"Data loaded. Number of UI samples: {len(ui_details_df)}")


Data loaded. Number of UI samples: 66261


In [33]:
# Define the image processing transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to the size expected by your model
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalization for pretrained models
])


In [36]:
# Load and preprocess images and labels
print("Loading and processing images...")
features = []
labels = []

Loading and processing images...


In [37]:
for index, row in ui_details_df.iterrows():
    ui_number = row['UI Number']
    is_social = row['Is_Social']
    img_path = os.path.join(images_dir, f"{ui_number}.jpg")

    if os.path.exists(img_path):
        image = Image.open(img_path).convert("RGB")  # Convert to RGB
        image_tensor = transform(image)
        features.append(image_tensor)
        labels.append(torch.tensor(is_social, dtype=torch.float32))
    else:
        print(f"Image {ui_number}.jpg not found.")

print(f"Loaded {len(features)} images successfully.")


Loaded 66261 images successfully.


In [38]:

# Create feature and label tensors
features_tensor = torch.stack(features) if features else torch.Tensor()
labels_tensor = torch.stack(labels) if labels else torch.Tensor()


In [39]:
# Create a dataset and dataloaders
dataset = TensorDataset(features_tensor, labels_tensor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64, shuffle=False)


In [40]:
# Define the model (Pretrained ResNet50)
print("Loading the ResNet50 model...")
model = models.resnet50(pretrained=True)


Loading the ResNet50 model...


/Users/wenpeishao/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/wenpeishao/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/wenpeishao/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████████████████████████████████| 97.8M/97.8M [00:03<00:00, 29.3MB/s]


In [41]:
# Modify the classifier for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1),
    nn.Sigmoid()
)

In [42]:
# Move the model to the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model = nn.DataParallel(model, device_ids=[0, 1, 2, 3])


In [43]:
# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)




In [44]:
# Training and evaluation loop
print("Starting training...")
num_epochs = 10
best_auc = 0.0
best_model_state = None


Starting training...


In [ ]:
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    for batch in train_loader:
        data, target = batch
        data = data.to(device)
        target = target.to(device).float().view(-1)  # Ensure target is a 1D float tensor

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output.view(-1), target)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs} - Training loss: {running_loss / len(train_loader):.4f}")

    # Evaluation phase
    model.eval()
    val_targets = []
    val_outputs = []
    with torch.no_grad():
        for batch in val_loader:
            data, target = batch
            data = data.to(device)
            target = target.to(device).float().view(-1)
            output = model(data)
            val_outputs.extend(output.view(-1).cpu().numpy())
            val_targets.extend(target.cpu().numpy())

    # Calculate AUC for the current epoch
    auc = roc_auc_score(val_targets, val_outputs)
    print(f"Epoch {epoch+1}/{num_epochs} - Validation AUC: {auc:.4f}")

    # Save best model based on AUC
    if auc > best_auc:
        best_auc = auc
        best_model_state = model.state_dict()
        print(f"Best model updated with AUC: {best_auc:.4f}")


In [ ]:

# Save the best model state to a file
if best_model_state is not None:
    torch.save(best_model_state, './best_resnet_finetune.pth')
    print(f"Best model saved with AUC: {best_auc:.4f}")

print("Training completed successfully.")
